# Why ITP_ID 200 isn't showing up

In [1]:
import pandas as pd
import os
#os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import calitp
from calitp.tables import tbl
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
(tbl.gtfs_schedule.stops()
# unique on itp_id and url_number
# but there are handful of agencies where same stop has multiple url's
 >> filter(_.calitp_itp_id == 200)
 >> arrange(_.calitp_itp_id, _.stop_id, _.calitp_url_number)
 >> collect()
)

,calitp_itp_id,calitp_url_number,stop_id,tts_stop_name,stop_lat,stop_lon,zone_id,parent_station,stop_code,stop_name,stop_desc,stop_url,location_type,stop_timezone,wheelchair_boarding,level_id,platform_code,calitp_extracted_at


In [4]:
(tbl.gtfs_schedule.trips()
# unique on itp_id and url_number
# but there are handful of agencies where same stop has multiple url's
 >> filter(_.calitp_itp_id == 200)
 >> arrange(_.calitp_itp_id, _.trip_id, _.calitp_url_number)
 >> collect()
)

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at


* Querying doesn't return ITP_ID==200 in `gtfs_schedule`
* But, in the `gtfs_schedule_history.calitp_files_updates` tables, the `stops` and `trips` tables are present

In [14]:
selected_files = ["stops.txt", "trips.txt"]

mtc = (
    tbl.gtfs_schedule_history.calitp_files_updates()
    >> filter(_.calitp_itp_id==200)
    >> select(_.calitp_itp_id, _.calitp_url_number, 
           _.calitp_extracted_at, _.name, _.size)
    >> arrange(_.calitp_extracted_at)
    >> collect()
    >> filter(_.name.isin(selected_files))
)

mtc = mtc.assign(
    date = pd.to_datetime(mtc.calitp_extracted_at)
)

In [12]:
mtc.head()

,calitp_itp_id,calitp_url_number,calitp_extracted_at,name,size
3,200,0,2021-07-31,stops.txt,1807999
20,200,0,2021-07-31,trips.txt,4927949
27,200,0,2021-08-01,stops.txt,1807999
48,200,0,2021-08-01,trips.txt,4927949
51,200,0,2021-08-03,stops.txt,1846170


In [16]:
print(f"min date: {mtc.date.min()}")
print(f"max date: {mtc.date.max()}")

min date: 2021-07-31 00:00:00
max date: 2021-12-15 00:00:00
